![](image.jpg)


Dive into the heart of data science with a project that combines healthcare insights and predictive analytics. As a Data Scientist at a top Health Insurance company, you have the opportunity to predict customer healthcare costs using the power of machine learning. Your insights will help tailor services and guide customers in planning their healthcare expenses more effectively.

## Dataset Summary

Meet your primary tool: the `insurance.csv` dataset. Packed with information on health insurance customers, this dataset is your key to unlocking patterns in healthcare costs. Here's what you need to know about the data you'll be working with:

## insurance.csv
| Column    | Data Type | Description                                                      |
|-----------|-----------|------------------------------------------------------------------|
| `age`       | int       | Age of the primary beneficiary.                                  |
| `sex`       | object    | Gender of the insurance contractor (male or female).             |
| `bmi`       | float     | Body mass index, a key indicator of body fat based on height and weight. |
| `children`  | int       | Number of dependents covered by the insurance plan.              |
| `smoker`    | object    | Indicates whether the beneficiary smokes (yes or no).            |
| `region`    | object    | The beneficiary's residential area in the US, divided into four regions. |
| `charges`   | float     | Individual medical costs billed by health insurance.             |



A bit of data cleaning is key to ensure the dataset is ready for modeling. Once your model is built using the `insurance.csv` dataset, the next step is to apply it to the `validation_dataset.csv`. This new dataset, similar to your training data minus the `charges` column, tests your model's accuracy and real-world utility by predicting costs for new customers.

## Let's Get Started!

This project is your playground for applying data science in a meaningful way, offering insights that have real-world applications. Ready to explore the data and uncover insights that could revolutionize healthcare planning? Let's begin this exciting journey!

In [457]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

# Loading the insurance dataset
insurance_data_path = 'insurance.csv'
insurance = pd.read_csv(insurance_data_path)
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19.0,female,27.900,0.0,yes,southwest,16884.924
1,18.0,male,33.770,1.0,no,Southeast,1725.5523
2,28.0,male,33.000,3.0,no,southeast,$4449.462
3,33.0,male,22.705,0.0,no,northwest,$21984.47061
4,32.0,male,28.880,0.0,no,northwest,$3866.8552


In [458]:
val_data = pd.read_csv('validation_dataset.csv')
val_data.head()

,age,sex,bmi,children,smoker,region
0,18.0,female,24.090000,1.0,no,southeast
1,39.0,male,26.410000,0.0,yes,northeast
2,27.0,male,29.150000,0.0,yes,southeast
3,71.0,male,65.502135,13.0,yes,southeast
4,28.0,male,38.060000,0.0,no,southeast


## Data Cleaning

In [459]:
insurance.dtypes

age         float64
sex          object
bmi         float64
children    float64
smoker       object
region       object
charges      object
dtype: object

In [460]:
insurance["age"].value_counts()

 18.0    63
 19.0    62
 48.0    29
 46.0    28
 53.0    27
         ..
-28.0     1
-25.0     1
-56.0     1
-58.0     1
-33.0     1
Name: age, Length: 80, dtype: int64

In [461]:
insurance["sex"].value_counts()

male      517
female    503
man        64
M          64
woman      62
F          62
Name: sex, dtype: int64

In [462]:
insurance["region"].value_counts()

Southeast    172
southeast    170
southwest    164
Northwest    161
Northeast    155
northeast    153
northwest    149
Southwest    148
Name: region, dtype: int64

In [463]:
insurance["charges"].value_counts()

$nan            12
1639.5631        2
16884.924        1
13555.0049       1
4661.28635       1
                ..
7345.084         1
26109.32905      1
$28287.89766     1
1149.3959        1
29141.3603       1
Name: charges, Length: 1272, dtype: int64

In [464]:
#Make all ages positive
#remove dollar signs, nans and make all charges floats
#make sex either M or F
#make region capitalization the same
insurance["age"] = insurance["age"].abs()

In [465]:
insurance["charges"] = insurance["charges"].str.replace("$", "")

In [466]:
sex_replace = []
for s in insurance["sex"]:
    if s=="M" or s=="man":
        sex_replace.append("male")
    elif s=="F" or s=="woman":
        sex_replace.append("female")
    else:
        sex_replace.append(s)
insurance["sex"] = sex_replace

In [467]:
insurance["region"] = insurance["region"].str.lower()

In [468]:
insurance = insurance.dropna()

In [469]:
insurance["charges"] = insurance["charges"].astype("float64")

In [470]:
insurance = insurance.dropna()

In [471]:
gender = pd.get_dummies(insurance["sex"], drop_first=True)
smoke = pd.get_dummies(insurance["smoker"], drop_first=True)
region = pd.get_dummies(insurance["region"], drop_first=True)

In [472]:
insurance = insurance.join(gender)
insurance = insurance.join(smoke)
insurance = insurance.join(region)

### Repeat data cleaning on validation set

In [473]:
val_data["age"] = val_data["age"].abs()
sex_replace = []
for s in val_data["sex"]:
    if s=="M" or s=="man":
        sex_replace.append("male")
    elif s=="F" or s=="woman":
        sex_replace.append("female")
    else:
        sex_replace.append(s)
val_data["sex"] = sex_replace
val_data["region"] = val_data["region"].str.lower()
val_data = val_data.dropna()
gender = pd.get_dummies(val_data["sex"], drop_first=True)
smoke = pd.get_dummies(val_data["smoker"], drop_first=True)
region = pd.get_dummies(val_data["region"], drop_first=True)
val_data = val_data.join(gender)
val_data = val_data.join(smoke)
val_data = val_data.join(region)
val_data = val_data.drop(["sex", "smoker", "region"], axis=1)

In [474]:
# scaler = StandardScaler()
# insurance["charges"] = scaler.fit_transform(insurance[["charges"]])

## Modeling

In [475]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression

In [476]:
X = insurance.drop(["charges", "sex", "smoker", "region"], axis=1)
y = insurance["charges"]

In [477]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Linear Regression

In [478]:
from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits=3, shuffle=True, random_state=42)
reg = LinearRegression()
cv_results = cross_val_score(reg, X, y, cv=kf, scoring='r2')
r2_score = cv_results.mean()
r2_score

0.7435348714289933

In [479]:
np.quantile(cv_results, [0.025, 0.975])

array([0.7120185 , 0.77128225])

In [480]:
reg.fit(X_train, y_train)

LinearRegression()

## Validation Set Predictions

In [481]:
preds = reg.predict(val_data)
clean_preds = []
for p in preds:
    if p < 1000:
        clean_preds.append(1000)
    else:
        clean_preds.append(p)

In [482]:
validation_data = val_data
validation_data["predicted_charges"] = clean_preds
validation_data


,age,bmi,children,male,yes,northwest,southeast,southwest,predicted_charges
0,18.0,24.090000,1.0,0,0,0,1,0,1000.000000
1,39.0,26.410000,0.0,1,1,0,0,0,31133.450387
2,27.0,29.150000,0.0,1,1,0,1,0,27965.611569
3,71.0,65.502135,13.0,1,1,0,1,0,55748.823085
4,28.0,38.060000,0.0,1,0,0,1,0,6867.894254
5,70.0,72.958351,11.0,0,1,0,1,0,57519.466773
6,29.0,32.110000,2.0,0,0,1,0,0,6969.325592
7,42.0,41.325000,1.0,0,0,0,0,0,13147.533125
8,48.0,36.575000,0.0,0,0,1,0,0,12878.024826
9,63.0,33.660000,3.0,1,0,0,1,0,15940.199456
